In [1]:
import numpy as np
import verde as vd
import harmonica as hm
import matplotlib.pyplot as plt
import xarray as xr

In [27]:
region = [0, 20e3, 0, 30e3, -15e3, 0]
spacing = 5e3
easting, northing = vd.grid_coordinates(region[:4], spacing=spacing, adjust="region", pixel_register=True)
_, vertical = vd.grid_coordinates(region[2:], spacing=spacing, adjust="region", pixel_register=True)
coordinates = easting[0, :], northing[:, 0], vertical[:, 0]
shape = tuple(c.size for c in reversed(coordinates))
dims = ("vertical", "northing", "easting")
mesh = xr.Dataset(
    {"density": (dims, np.arange(np.product(shape)).reshape(shape))}, 
    dict((name, c) for name, c in zip(dims, reversed(coordinates))),
)
mesh.attrs["cell_size"] = tuple(mesh[dim][1].values - mesh[dim][0].values for dim in dims)
mesh.attrs["region"] = region
mesh

<xarray.Dataset>
Dimensions:   (vertical: 3, northing: 6, easting: 4)
Coordinates:
  * vertical  (vertical) float64 -1.25e+04 -7.5e+03 -2.5e+03
  * northing  (northing) float64 2.5e+03 7.5e+03 1.25e+04 ... 2.25e+04 2.75e+04
  * easting   (easting) float64 2.5e+03 7.5e+03 1.25e+04 1.75e+04
Data variables:
    density   (vertical, northing, easting) int64 0 1 2 3 4 5 ... 67 68 69 70 71
Attributes:
    cell_size:  (5000.0, 5000.0, 5000.0)
    region:     [0, 20000.0, 0, 30000.0, -15000.0, 0]

In [42]:
def get_prism(index, mesh):
    center = [mesh[dim].values[i] for i, dim in zip(index, mesh.dims)]
    bounds = []
    for c, size in reversed(list(zip(center, mesh.attrs["cell_size"]))):
        bounds.extend([c - size/2, c + size/2])
    density = mesh.density.values[index]
    return bounds, density

get_prism((1, 0, 1), mesh)

([5000.0, 10000.0, 0.0, 5000.0, -10000.0, -5000.0], 25)